Imports

In [65]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
from datetime import date

Athlete Class

In [66]:
class Athlete: 
    def __init__(self, id, name, training_plan, current_weight, competition_weight_category, competitions_entered, private_coaching_hours, notes, itemised_monthly_costs=None, total_training_and_comp_costs=None, weights=None):
        self.id = id
        self.name = name
        self.training_plan = training_plan
        self.current_weight = current_weight
        self.competition_weight_category = competition_weight_category
        self.competitions_entered = competitions_entered
        self.private_coaching_hours = private_coaching_hours
        self.notes = notes

        self.itemised_monthly_costs = itemised_monthly_costs
        self.total_training_and_comp_costs = total_training_and_comp_costs
        self.weights = weights

Data

In [67]:
weight_categories = {
    # Category & Upper Limit
        "Heavyweight": 999,
        "Light-Heavyweight": 100,
        "Middleweight": 90,
        "Light-Middleweight": 81,
        "Lightweight": 73,
        "Flyweight": 66}

Validations

In [68]:
# Validate training plan entry
valid_plans = ["Beginner", "Intermediate", "Elite"]

# Validate weight category entry box for create new athlete !!!! POSSIBLE ERROR IN CODE !!!!
valid_categories = ["Heavyweight", "Light-Heavyweight", "Middleweight", "Light-Middleweight", "Lightweight", "Flyweight"]

Global Functions

In [69]:
def open_weight_cats(): 
    # Create a new top-level window
    weight_cats_help_window = tk.Toplevel()  
    weight_cats_help_window.title("Weight Categories Help")  

    # Create a frame to contain the content with padding and border
    content_frame = tk.Frame(weight_cats_help_window, padx=20, pady=20, bd=2, relief="groove")
    content_frame.pack(padx=10, pady=10)

    # Header labels
    tk.Label(content_frame, text="Category", width=20, anchor="w", font=("Helvetica", 12, "bold")).grid(row=0, column=0, padx=10, pady=5)
    tk.Label(content_frame, text="Weight Limit (Kg)", width=20, anchor="e", font=("Helvetica", 12, "bold")).grid(row=0, column=1, padx=10, pady=5)

    # Adding a line separator at the top
    tk.Frame(content_frame, height=2, width=400, bg="black").grid(row=1, columnspan=2, pady=5)

    # Iterate over weight_categories dictionary (Key/Value pairs)
    for idx, (category, limit) in enumerate(weight_categories.items(), start=2):
        # Labels for category and limit
        tk.Label(content_frame, text=category, width=20, anchor="w").grid(row=idx, column=0, padx=10, pady=5, sticky="w")
        tk.Label(content_frame, text=limit, width=20, anchor="e").grid(row=idx, column=1, padx=10, pady=5, sticky="e")

Main (App) Class

In [70]:
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("N S J")

        # Set Window Dimensions & Center on Users Screen
        width = 750
        height = 210
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        x_coordinate = (screen_width - width) // 2
        y_coordinate = (screen_height - height) // 2
        root.geometry(f"{width}x{height}+{x_coordinate}+{y_coordinate}")

        # Create a new frame for the content with a border
        content_frame = tk.Frame(root, bd=2, relief="groove", )
        content_frame.pack(padx=20, pady=20)

        # Title (H1) inside the content frame
        title_label = tk.Label(content_frame, text="Athlete Management Portal", font=("Helvetica", 24, "bold"))
        title_label.pack(pady=10)

        # Database connection
        self.conn = sqlite3.connect("Athletes.db")
        self.c = self.conn.cursor()

        # Frame & Buttons
        root_window_button_frame = tk.Frame(content_frame,)
        root_window_button_frame.pack(pady=20)

        # Create Athlete Button
        trainer_button = tk.Button(root_window_button_frame, text="Add new Athlete", command = self.create_athlete)
        trainer_button.grid(row=0, column=0, padx=(50, 30), sticky="ew")

        # View all Athletes Button
        athlete_button = tk.Button(root_window_button_frame, text="Manage Athletes", command = self.view_records)
        athlete_button.grid(row=0, column=2, padx=30, sticky="ew")

        # Competition Calendar Button
        comp_calendar_button = tk.Button(root_window_button_frame, text="Competition Calendar", command = self.competition_calendar)
        comp_calendar_button.grid(row=1, column=1, padx=(30, 50), sticky="ew")

        # Grid weight configuration to make buttons the correct size
        root_window_button_frame.columnconfigure(0, weight=1)
        root_window_button_frame.columnconfigure(1, weight=1)
        root_window_button_frame.columnconfigure(2, weight=1)

        # initialise the creation of the athletes table (IF it doesn't already exist)
        self.create_table()


    def create_table(self):
        self.c.execute("""CREATE TABLE IF NOT EXISTS athletes (
                            id INTEGER PRIMARY KEY,
                            name TEXT,
                            training_plan TEXT,
                            current_weight REAL,
                            competition_weight_category TEXT,
                            competitions_entered INTEGER,
                            private_coaching_hours INTEGER,
                            itemised_monthly_costs INTEGER,
                            total_training_and_comp_costs INTEGER,
                            current_weight_vs_comp_weight INTEGER,

                            notes TEXT
                        )""")
        self.conn.commit()


    def view_records(self):
        # Create the view_records_window
        view_records_window = tk.Toplevel(self.root)
        view_records_window.title("View Records")

        # Set Window Dimensions & Center on Users Screen
        width = 300
        height = 275
        screen_width = view_records_window.winfo_screenwidth()
        screen_height = view_records_window.winfo_screenheight()
        x_coordinate = (screen_width - width) // 2
        y_coordinate = (screen_height - height) // 2
        view_records_window.geometry(f"{width}x{height}+{x_coordinate}+{y_coordinate}")

        # Label for athlete selection
        select_label = tk.Label(view_records_window, text="Select an Athlete to manage")
        select_label.pack(pady=(10, 5))

        # Fetch all records from the "athletes" table
        self.c.execute("SELECT * FROM athletes")
        athletes = self.c.fetchall()

        # Variable to store the selected athlete
        selected_athlete = tk.StringVar()

        # Create a frame to contain radio buttons with increased padding
        radio_frame = tk.Frame(view_records_window, bd=2, relief="groove")
        radio_frame.pack(pady=10, padx=10)

        # Display athletes in a list format
        for id, athlete in enumerate(athletes):
            id, name, _, _, _, _, _, _, _, _, _ = athlete
            # Create a radio button for each athlete
            radio_button = tk.Radiobutton(radio_frame, text=f"{name} ", value=id, variable=selected_athlete, anchor="e")
            radio_button.pack(anchor="w", padx=10, pady=(2, 2))

        def view_selected_athlete():
            selected_athlete_id = selected_athlete.get()

            if selected_athlete_id:
                view_selected_athlete_window = tk.Toplevel(view_records_window)
                view_selected_athlete_window.title("View Selected Athlete")
                frame = tk.Frame(view_selected_athlete_window, padx=20, pady=20, bd=2, relief="groove")
                frame.pack(padx=20, pady=20)

                self.c.execute("SELECT * FROM athletes WHERE id=?", (selected_athlete_id,))
                selected_athlete_data = self.c.fetchone()

                if selected_athlete_data:
                    labels = [
                        "Name",
                        "Training Plan",
                        "Current Weight (Kg)",
                        "Comp Weight Category",
                        "# of Competitions Entered This Month",
                        "# of Private Coaching Hrs This Month",
                        "Itemised Monthly Costs",
                        "Total Training And Comp Costs",
                        "Current Weight Vs Comp Weight"
                    ]

                    entry_fields = []  # Define entry_fields list

                    # Insert data for existing labels and entries
                    for idx, label_text in enumerate(labels):
                        label = tk.Label(frame, text=label_text)
                        label.grid(row=idx, column=0, padx=10, pady=5, sticky="w")
                        entry = tk.Entry(frame)
                        entry.grid(row=idx, column=1, padx=10, pady=5, sticky="e")

                        try:
                            entry_value = str(selected_athlete_data[idx + 1])  # Convert data to string
                            entry.insert(tk.END, entry_value)  # Insert data from selected_athlete_data
                        except IndexError:
                            print(f"IndexError: selected_athlete_data does not have index {idx + 1}")
                        except Exception as e:
                            print(f"Unexpected error: {e}")

                        entry_fields.append(entry)  # Add Entry widget to entry_fields list

                    # Additional calculations and updates for specific fields
                    level = selected_athlete_data[2].lower()  # Made lower to circumvent possible input errors
                    current_weight = float(selected_athlete_data[3])
                    comp_weight_category = selected_athlete_data[4]
                    competitions_entered = int(selected_athlete_data[5])
                    private_coaching_hours = int(selected_athlete_data[6])

                    # Calculate Itemised Monthly Costs
                    if level == 'beginner':
                        total_training_cost = (2 * 25) * 4
                    elif level == 'intermediate':
                        total_training_cost = (3 * 30) * 4
                    elif level == 'elite':
                        total_training_cost = (5 * 35) * 4
                    else:
                        total_training_cost = 0

                    coaching_cost = private_coaching_hours * 9.5
                    competition_cost = competitions_entered * 22
                    itemised_monthly_costs = f"""
                    Training this month: £{total_training_cost:.2f}
                    ,Total coaching cost: £{coaching_cost:.2f} at {private_coaching_hours} hours, Competitions entered this month: {competitions_entered} at £{competition_cost:.2f} cost
                    """
                    total_training_and_comp_costs = total_training_cost + competition_cost

                    # Calculate Current Weight Vs Comp Weight
                    comp_weight_category_limit = get_comp_weight_limit(comp_weight_category)
                    current_vs_comp_weight = f"{current_weight} Kg vs Comp weight limit of {comp_weight_category_limit} Kg"

                    # Updating Specific Fields with Calculated Values

                    entry_fields[6].delete(0, tk.END)
                    entry_fields[6].insert(tk.END, itemised_monthly_costs.strip())

                    entry_fields[7].delete(0, tk.END)
                    entry_fields[7].insert(tk.END, f"£{total_training_and_comp_costs:.2f}")

                    entry_fields[8].delete(0, tk.END)
                    entry_fields[8].insert(tk.END, current_vs_comp_weight)

                    # Notes section
                    notes_label = tk.Label(frame, text="Notes", anchor="w")
                    notes_label.grid(row=0, column=2, columnspan=2, padx=10, pady=5)
                    notes_entry = tk.Text(frame, height=10, width=40)
                    notes_entry.grid(row=1, column=2, rowspan=6, padx=10, pady=5, sticky="nsew")
                    try:
                        notes_entry.insert(tk.END, str(selected_athlete_data[len(labels) + 1]))  # Corrected index
                    except IndexError:
                        print(f"IndexError: selected_athlete_data does not have index {len(labels) + 1}")
                    except Exception as e:
                        print(f"Unexpected error when inserting notes: {e}")

                    def update_record():
                        updated_data = [entry.get() for entry in entry_fields]
                        updated_data.append(notes_entry.get("1.0", tk.END).strip())
                        try:
                            self.c.execute("""UPDATE athletes SET name=?, training_plan=?, current_weight=?, 
                                            competition_weight_category=?, competitions_entered=?, 
                                            private_coaching_hours=?, itemised_monthly_costs=?, total_training_and_comp_costs=?, current_weight_vs_comp_weight=?, notes=? WHERE id=?""",
                                            (*updated_data, selected_athlete_id))
                            self.conn.commit()
                            messagebox.showinfo("Success", "Record updated successfully!")
                            view_selected_athlete_window.destroy()
                            view_records_window.destroy()
                        except Exception as e:
                            print(f"Unexpected error during update: {e}")
                            messagebox.showerror("Error", "Failed to update the record.")

                    def delete_record():
                        confirm = messagebox.askyesno("Delete Record", "Are you sure you want to delete this record?")
                        if confirm:
                            try:
                                self.c.execute("DELETE FROM athletes WHERE id=?", (selected_athlete_id,))
                                self.conn.commit()
                                messagebox.showinfo("Success", "Record deleted successfully!")
                                view_selected_athlete_window.destroy()
                                view_records_window.destroy()
                            except Exception as e:
                                print(f"Unexpected error during delete: {e}")
                                messagebox.showerror("Error", "Failed to delete the record.")

                    delete_button = tk.Button(view_selected_athlete_window, text="Delete Record", command=delete_record)
                    delete_button.pack(side="left", padx=(20, 5), pady=(5, 20), anchor="sw")
                    update_button = tk.Button(view_selected_athlete_window, text="Update Record", command=update_record)
                    update_button.pack(side="right", padx=(5, 20), pady=(5, 20), anchor="se")

        # Back Button
        back_button = tk.Button(view_records_window, text="Back", command=view_records_window.destroy)
        back_button.pack(side="left", anchor="se", padx=10, pady=(5, 10))  # Adjusted pady from 10 to (5, 10)

        # Button to view athlete details
        view_selected_athlete_button = tk.Button(view_records_window, text="View Athlete", command=view_selected_athlete)
        view_selected_athlete_button.pack(side="right", anchor="sw", padx=10, pady=(5, 10))  # Adjusted pady from 10 to (5, 10)

        # Update window dimensions after adding all elements (So it automatically sizes for the amount of entries in the DB)
        view_records_window.update_idletasks()
        width = view_records_window.winfo_reqwidth() + 20  # Add padding
        height = view_records_window.winfo_reqheight() + 20  # Add padding
        screen_width = view_records_window.winfo_screenwidth()
        screen_height = view_records_window.winfo_screenheight()
        x_coordinate = (screen_width - width) // 2
        y_coordinate = (screen_height - height) // 2
        view_records_window.geometry(f"{width}x{height}+{x_coordinate}+{y_coordinate}")

        # "Helper" function to get competition weight limit
        def get_comp_weight_limit(category):
            for cat, limit in weight_categories.items():
                if cat == category:
                    return limit
            return 0


    def create_athlete(self):
        self.create_athlete_window = tk.Toplevel(self.root)  # Store a reference to the window for alter use
        self.create_athlete_window.title("Create Athlete")

        # Create a frame for labels and entry fields with border and padding
        frame = tk.Frame(self.create_athlete_window, padx=20, pady=20, bd=2, relief="groove")
        frame.pack(padx=20, pady=20)

        # Labels
        tk.Label(frame, text="Athlete Name").grid(row=0, column=0, sticky="w", padx=10, pady=5)
        tk.Label(frame, text="Training Plan").grid(row=1, column=0, sticky="w", padx=10, pady=5)
        tk.Label(frame, text="Current Weight (Kg)").grid(row=2, column=0, sticky="w", padx=10, pady=5)
        tk.Label(frame, text="Competition Weight Category").grid(row=3, column=0, sticky="w", padx=10, pady=5)
        tk.Label(frame, text="Competitions Entered This Month").grid(row=4, column=0, sticky="w", padx=10, pady=5)
        tk.Label(frame, text="Private Coaching Hours This Month").grid(row=5, column=0, sticky="w", padx=10, pady=5)

        tk.Label(frame, text="Notes", anchor="w").grid(row=0, column=2, columnspan=2, padx=10, pady=5)

        # Entry fields
        self.name_entry = tk.Entry(frame)
        self.name_entry.grid(row=0, column=1, padx=10, pady=5)
        self.training_plan_entry = tk.Entry(frame)
        self.training_plan_entry.grid(row=1, column=1, padx=10, pady=5)
        self.weight_entry = tk.Entry(frame)
        self.weight_entry.grid(row=2, column=1, padx=10, pady=5)
        self.category_entry = tk.Entry(frame)
        self.category_entry.grid(row=3, column=1, padx=10, pady=5)
        self.competitions_entry = tk.Entry(frame)
        self.competitions_entry.grid(row=4, column=1, padx=10, pady=5)
        self.coaching_hours_entry = tk.Entry(frame)
        self.coaching_hours_entry.grid(row=5, column=1, padx=10, pady=5)
        self.notes_text = tk.Text(frame, height=10, width=40)
        self.notes_text.grid(row=1, column=2, rowspan=6, padx=10, pady=5, sticky="nsew")

        # "Save Record" button outside the frame with reduced padding at the top
        tk.Button(self.create_athlete_window, text="Save Record", command=self.save_record).pack(side="right", padx=20, pady=(10, 20), anchor="se")

        # "Weight Categories" help button outside the frame
        tk.Button(self.create_athlete_window, text="Weight Categories?", command=open_weight_cats).pack(side="left", padx=20, pady=(10, 20), anchor="sw")


    def save_record(self):
        # Retrieve data from entry fields
        name = self.name_entry.get()
        training_plan = self.training_plan_entry.get().capitalize()  # Capitalise input for consistency
        weight = self.weight_entry.get()
        category = self.category_entry.get()  # .capitalise input for consistency removed as causing errors
        competitions = self.competitions_entry.get()
        coaching_hours = self.coaching_hours_entry.get()
        notes = self.notes_text.get("1.0", tk.END).strip()

        # !!!! LOGIC CHECKS !!!

        # Check if any field is empty
        if not all([name, training_plan, weight, category, competitions, coaching_hours]):
            messagebox.showerror("Error", "Please fill in all fields.")
            return

        # Check if training plan is valid
        if training_plan not in valid_plans:
            messagebox.showerror("Error", f"Invalid training plan '{training_plan}'. Please enter one of: {', '.join(valid_plans)}.")
            return

        # Check if weight category is valid
        if category not in valid_categories:
            messagebox.showerror("Error", f"Invalid weight category '{category}'. Please enter one of: {', '.join(valid_categories)}.")
            return

        # Check if weight is a valid float
        try:
            weight = float(weight)
        except ValueError:
            messagebox.showerror("Error", "Invalid weight format. Please enter a valid number.")
            return

        # Check if competitions entered is a valid integer
        try:
            competitions = int(competitions)
        except ValueError:
            messagebox.showerror("Error", "Invalid competitions format. Please enter a valid integer.")
            return

        # Check if coaching hours entered is a valid integer
        try:
            coaching_hours = int(coaching_hours)
            if coaching_hours > 5 * 4:  # 5 hours per week * 4 weeks = 20
                messagebox.showerror("Error", "Private coaching hours exceed the maximum limit (20 hours).")
                return
        except ValueError:
            messagebox.showerror("Error", "Invalid coaching hours format. Please enter a valid integer.")
            return

        # Prevent new athletes from entering a competition if not on Intermediate or Elite training plan
        if competitions > 0 and training_plan not in ["Intermediate", "Elite"]:
            messagebox.showerror("Error", "Athletes must be on an Intermediate or Elite training plan to enter competitions.")
            return

        # Insert data into database
        self.c.execute("""INSERT INTO athletes (name, training_plan, current_weight, 
                        competition_weight_category, competitions_entered, private_coaching_hours, notes)
                        VALUES (?, ?, ?, ?, ?, ?, ?)""",
                        (name, training_plan, weight, category, competitions, coaching_hours, notes))
        self.conn.commit()

        # Clear entry fields
        self.name_entry.delete(0, tk.END)
        self.training_plan_entry.delete(0, tk.END)
        self.weight_entry.delete(0, tk.END)
        self.category_entry.delete(0, tk.END)
        self.competitions_entry.delete(0, tk.END)
        self.coaching_hours_entry.delete(0, tk.END)
        self.notes_text.delete("1.0", tk.END)

        # Destroy the create_athlete_window
        self.create_athlete_window.destroy()

        # Show success message
        messagebox.showinfo("Record Added", f"Athlete: {name}")


    def competition_calendar(self):
            class CalendarApp:
                MONTH_NAMES = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
                DAY_NAMES = ["Saturday", "Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

                def __init__(self, window):
                    self.window = window
                    self.month = date.today().month
                    self.year = date.today().year
                    self.conn = sqlite3.connect('Calendar.db')
                    self.c = self.conn.cursor()
                    self.create_database()
                    self.create_widgets()

                def create_database(self):
                    self.c.execute('''CREATE TABLE IF NOT EXISTS calendar (
                                        id INTEGER PRIMARY KEY,
                                        day INTEGER,
                                        month INTEGER,
                                        year INTEGER,
                                        text TEXT
                                    )''')
                    self.conn.commit()

                def create_widgets(self):
                    self.create_top_frame()
                    self.create_calendar_frame()
                    self.create_bottom_frame()
                    self.print_month_year()
                    self.make_buttons()
                    self.month_generator(self.day_month_starts(), self.days_in_month())

                def create_top_frame(self):
                    self.top_frame = tk.Frame(self.window, borderwidth=2, padx=10, pady=10)
                    self.top_frame.grid(row=0, column=0, sticky="ew")
                    self.top_frame.columnconfigure(0, weight=1)
                    self.top_frame.columnconfigure(2, weight=1)

                def create_calendar_frame(self):
                    self.calendar_frame = tk.Frame(self.window, borderwidth=2, relief="solid", pady=10, padx=30)  
                    self.calendar_frame.grid(row=1, column=0, sticky="ew")
                    self.window.columnconfigure(0, weight=1)

                def create_bottom_frame(self):
                    self.bottom_frame = tk.Frame(self.window, borderwidth=2, padx=10, pady=10)
                    self.bottom_frame.grid(row=2, column=0, sticky="ew")
                    self.bottom_frame.columnconfigure(0, weight=1)
                    self.bottom_frame.columnconfigure(2, weight=1)

                    self.load_button = tk.Button(self.bottom_frame, text="Load Entries", command=self.load_entries)
                    self.load_button.grid(row=0, column=0, sticky="w", padx=(0, 10))
                    self.save_button = tk.Button(self.bottom_frame, text="Save Entries", command=self.save_entry)
                    self.save_button.grid(row=0, column=2, sticky="e", padx=(10, 0))

                def print_month_year(self):
                    written_month = self.MONTH_NAMES[self.month - 1]
                    month_year_label = tk.Label(self.calendar_frame, text=f"{written_month} {self.year}", font=("Arial", 20))
                    month_year_label.grid(column=2, row=0, columnspan=3)

                def make_buttons(self):
                    go_back_button = tk.Button(self.top_frame, text="<", command=lambda: self.switch_months(-1))
                    go_back_button.grid(row=0, column=0, sticky="w")
                    go_forward_button = tk.Button(self.top_frame, text=">", command=lambda: self.switch_months(1))
                    go_forward_button.grid(row=0, column=2, sticky="e")

                def month_generator(self, start_date, num_days):
                    for name_number, name in enumerate(self.DAY_NAMES):
                        names = tk.Label(self.calendar_frame, text=name, fg="black")
                        names.grid(column=name_number, row=1, sticky='nsew')

                    index = 0
                    day = 1
                    self.text_objects = {}
                    for row in range(6):
                        for column in range(7):
                            if index >= start_date and index <= start_date + num_days - 1:
                                day_frame = tk.Frame(self.calendar_frame)
                                text_box = tk.Text(day_frame, width=15, height=5)
                                text_box.grid(row=1)
                                self.text_objects[day] = text_box
                                day_frame.grid(row=row + 2, column=column, sticky='nsew')
                                day_frame.columnconfigure(0, weight=1)
                                day_number_label = tk.Label(day_frame, text=day)
                                day_number_label.grid(row=0)
                                day += 1
                            index += 1

                def switch_months(self, direction):
                    if self.month == 12 and direction == 1:
                        self.month = 1
                        self.year += 1
                    elif self.month == 1 and direction == -1:
                        self.month = 12
                        self.year -= 1
                    else:
                        self.month += direction

                    self.recreate_calendar_frame()

                def recreate_calendar_frame(self):
                    self.calendar_frame.destroy()
                    self.create_calendar_frame()
                    self.print_month_year()
                    self.make_buttons()
                    self.month_generator(self.day_month_starts(), self.days_in_month())

                def save_entry(self):
                    try:
                        for day in range(1, len(self.text_objects) + 1):
                            text = self.text_objects[day].get("1.0", "end - 1 chars")
                            self.c.execute('''INSERT INTO calendar (day, month, year, text) VALUES (?, ?, ?, ?)''', (day, self.month, self.year, text))
                        self.conn.commit()
                    except Exception as e:
                        print(f"Error saving data: {e}")

                def load_entries(self):
                    try:
                        self.c.execute('''SELECT day, text FROM calendar WHERE month=? AND year=?''', (self.month, self.year))
                        data = self.c.fetchall()
                        for day, text in data:
                            self.text_objects[day].insert("1.0", text)
                    except Exception as e:
                        print(f"Error loading data: {e}")

                def is_leap_year(self, year):
                    if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0):
                        return True
                    else:
                        return False

                def day_month_starts(self):
                    last_two_year = self.year % 100
                    calculation = last_two_year // 4
                    calculation += 1
                    if self.month == 1 or self.month == 10:
                        calculation += 1
                    elif self.month == 2 or self.month == 3 or self.month == 11:
                        calculation += 4
                    elif self.month == 5:
                        calculation += 2
                    elif self.month == 6:
                        calculation += 5
                    elif self.month == 8:
                        calculation += 3
                    elif self.month == 9 or self.month == 12:
                        calculation += 6
                    else:
                        calculation += 0
                    leap_year = self.is_leap_year(self.year)
                    if leap_year and (self.month == 1 or self.month == 2):
                        calculation -= 1
                    calculation += 6
                    calculation += last_two_year
                    day_of_week = calculation % 7
                    return day_of_week

                def days_in_month(self):
                    # All months that have 31 days
                    if self.month in [1, 3, 5, 7, 8, 10, 12]:
                        return 31
                    # All months that have 30 days
                    elif self.month in [4, 6, 9, 11]:
                        return 30
                    else:
                        leap_year = self.is_leap_year(self.year)
                        if leap_year:
                            return 29
                        else:
                            return 28

            # Create the root window
            window = tk.Tk()
            window.title("Calendar")

            # Create the calendar app instance
            app = CalendarApp(window)

            window.mainloop()

            # Close SQLite connection when program exits
            app.conn.close()

In [71]:
def __del__(self):
        # Close database connection
        self.conn.close()

Close Loop

In [72]:
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()